In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import NullFormatter
from matplotlib.dates import MonthLocator, DateFormatter
import seaborn as sns

In [3]:
#defect data
df = pd.read_excel(r"C:\Users\Darius\OneDrive - SMRT Corporation Ltd\RCM\RCM TEMPLATE USE THIS FOR UPDATE_110221\Control charts CCLRS Template.xlsx", date_parser = "Record")

#train mileage data
mileage = pd.read_excel(r"C:\Users\Darius\OneDrive - SMRT Corporation Ltd\RCM\RCM TEMPLATE USE THIS FOR UPDATE_110221\Control charts CCLRS Template.xlsx", sheet_name = "Mileage" ,date_parser = "Record")

#train withdrawal data
pvw = pd.read_excel(r"C:\Users\Darius\OneDrive - SMRT Corporation Ltd\RCM\RCM TEMPLATE USE THIS FOR UPDATE_110221\Control charts CCLRS Template.xlsx", sheet_name = "Ops Data" ,date_parser = "Date")

### Data Cleaning

In [4]:
df["Fault Group"]  = df["Fault Group"].fillna("Nil")

In [5]:
#remove non-critical categories
df  = df.loc[~df["Fault Group"].str.contains(
    "Miscellaneous|Nil|pm|PM|Train|WIN System")]

In [6]:
#fill variable that is na in cause column
df.Cause.fillna('Uncategorised', inplace=True)

#remove non-critical cause
df = df.loc[~df["Cause"].str.contains(
    'S-ATC-Onboard-PISF/DR|No action done. System all in normal condition|No fault found (Deleted)|Nil|Not A Defect. No action needed|#N/A|nil|Nil|Uncategorised|Miscellaneous')].copy()

C:\Users\Darius\AppData\Local\Temp/ipykernel_1772/2933923013.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  df = df.loc[~df["Cause"].str.contains(


In [19]:
df.columns = df.columns.str.strip()

In [20]:
df  = df.loc[:,["Record","Order Number","Notification Number","Record_MonthName","WO Text","Fault Group",
                "Cause","FDS Remarks","Operation Text","Train Number","Fleet"]]

In [21]:
#further filter for control chart visualisation
dff  = df.loc[:,["Record_MonthName","Fault Group","Fleet","Train Number"]].copy()

In [22]:
dfpt = dff.pivot_table(index = ["Fleet","Record_MonthName"],columns = "Fault Group", aggfunc = "count" )
dfpt.fillna(0.0, inplace=True)
#dfpt.reset_index(level = 0, inplace=True)
#dfpt.sort_values(by = "Record_MonthName", ascending = False,inplace=True)

### filter by fleet (C830)

In [23]:
#filter by C830
dfptc830 = dfpt.loc[[dfpt.index.get_level_values(0)[1]]]

#reset index
dfptc830 = dfptc830.reset_index()

#drop fleet columns
dfptc830 = dfptc830.drop('Fleet', 1).copy()

#compress columns name
dfptc830.columns = dfptc830.columns.map('_'.join)

#set month as index
dfptc830 = dfptc830.set_index('Record_MonthName_')

#replace all the zero count as 0.1 to prevent NaN error.
dfptc830 = dfptc830.replace(0.0,0.1)

#C830 mileage
mileage.rename(columns={'Record': 'Record_MonthName_'}, inplace=True)
mileagec830 = mileage.iloc[:,[0,67]]
mileagec830["C830 (monthly)"] = mileagec830["C830 (monthly)"].fillna(1)
mileagec830 = mileagec830.set_index('Record_MonthName_')

#merge mileage and dfptc830 together
dfptc830 = pd.merge(dfptc830,mileagec830, how = "left", left_index=True, right_index=True )

#calculate defect rate of each fault category
a = dfptc830.iloc[:,:-1]
#a = dfptc830.iloc[:,0]
b = dfptc830.iloc[:,-1:]

a = a.fillna(0.1)
b = b.fillna(0.1)

c830_defectrate = a.div(b.values, axis=0) * 100000.0
c830_defectrate = c830_defectrate.round(2)

#calculate mean of each fault category
c830_mean = c830_defectrate.tail(18)
c830_mean = c830_mean.replace("Train Number_","mean")
c830_mean.reset_index(inplace=True)
c830_mean.columns = c830_mean.columns.str.replace("Train Number_","Mean ")

#calculate std *3 (UCL) of each fault category
c830_std = (c830_defectrate.rolling(18).std().tail(18) * 1.645) + (c830_defectrate.rolling(18).mean().tail(18))
c830_std = c830_std.replace("Train Number_","UCL")
c830_std.reset_index(inplace=True)
c830_std.columns = c830_std.columns.str.replace("Train Number_","UCL ")

#create a function to fix UCL line by year

def fixed_UCL(std):
        i = 0
        for i in range(len(std)):
            try:
                if std.Record_MonthName_.dt.month[i] != 12:
                    a = std.iloc[i,1:].copy()
                    std.iloc[i+1,1:] = a
            except IndexError :
                pass

##fix c830 UCL line by year
fixed_UCL(c830_std)

#merge mean data and std data
c830 = pd.merge(c830_mean,c830_std, how = "left",on  ="Record_MonthName_")
#round the decimal points
c830 = c830.round(2)

C:\Users\Darius\AppData\Local\Temp/ipykernel_1772/1344093881.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dfptc830 = dfptc830.drop('Fleet', 1).copy()
C:\Users\Darius\anaconda3\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
C:\Users\Darius\AppData\Local\Temp/ipykernel_1772/1344093881.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mileagec830["C830 (monthly)"] = mileagec830["C830 (monthly)"].fillna(1)


In [24]:
c830.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 0 to 17
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Record_MonthName_           18 non-null     datetime64[ns]
 1   Mean ATC System             18 non-null     float64       
 2   Mean Aircon System          18 non-null     float64       
 3   Mean Auxiliary System       18 non-null     float64       
 4   Mean Brake System           18 non-null     float64       
 5   Mean Coupler System         18 non-null     float64       
 6   Mean Electrical System      18 non-null     float64       
 7   Mean Fire Detection System  18 non-null     float64       
 8   Mean Lighting System        18 non-null     float64       
 9   Mean Pneumatic System       18 non-null     float64       
 10  Mean Propulsion System      18 non-null     float64       
 11  Mean Saloon Door System     18 non-null     float64       
 

In [25]:
#melt defect rate columns
c830dr = pd.melt(c830, id_vars=['Record_MonthName_'], value_vars=c830.iloc[:,1:12],var_name='Fault_Category',value_name='Defect_Rate', ignore_index=False)
#generalise fault category labels
c830dr.Fault_Category = c830dr.Fault_Category.str.replace("Mean","").str.strip()

#melt defect rate columns
c830ucl = pd.melt(c830, id_vars=['Record_MonthName_'], value_vars=c830.iloc[:,12:],var_name='Fault_Category',value_name='UCL', ignore_index=False)

#concat c830dr and c830ucl
c830dr["UCL"] = c830ucl.iloc[:,2]
c830 = c830dr
c830.reset_index(drop=True, inplace=True)

In [26]:
c830test = c830.copy()

In [27]:
c830test

,Record_MonthName_,Fault_Category,Defect_Rate,UCL
0,2020-12-01,ATC System,1.57,4.23
1,2021-01-01,ATC System,2.13,4.22
2,2021-02-01,ATC System,2.37,4.22
3,2021-03-01,ATC System,4.09,4.22
4,2021-04-01,ATC System,2.72,4.22
...,...,...,...,...
193,2022-01-01,Saloon Door System,2.94,3.48
194,2022-02-01,Saloon Door System,1.24,3.48
195,2022-03-01,Saloon Door System,2.21,3.48
196,2022-04-01,Saloon Door System,0.71,3.48


In [28]:
#check for defect rate above UCL
x=0
c830test["above_UCL"] = ""

for x in range(len(c830test.loc[:,"Defect_Rate"])):
    if c830test.loc[x,"Defect_Rate"] >= c830test.loc[x,"UCL"]:
        c830test.loc[x,"above_UCL"] = c830test.loc[x,"Defect_Rate"]
    else:
        c830test.loc[x,"above_UCL"] = 0

In [29]:
x=0
y=0
#loop to capture all upward trends (3 consecutive upward trend)

for y in range(c830test.Fault_Category.nunique()):
    for x in range(18):
        if (x+3+(18*y) == 198):
            break
        elif (c830test.loc[x+1+(18*y),"Fault_Category"] == c830test.Fault_Category.unique()[y])|(
            c830test.loc[x+2+(18*y),"Fault_Category"] == c830test.Fault_Category.unique()[y])|(
            c830test.loc[x+3+(18*y),"Fault_Category"] == c830test.Fault_Category.unique()[y]):

            if (c830test.loc[x+(18*y),"Defect_Rate"] < c830test.loc[x+1+(
                18*y),"Defect_Rate"] < c830test.loc[x+2+(18*y),"Defect_Rate"] < c830test.loc[x+3+(18*y),"Defect_Rate"]):

                    c830test.loc[x+(18*y),"upward_trend"] = c830test.loc[x+(18*y),"Defect_Rate"]
                    c830test.loc[x+1+(18*y),"upward_trend"] = c830test.loc[x+1+(18*y),"Defect_Rate"]
                    c830test.loc[x+2+(18*y),"upward_trend"] = c830test.loc[x+2+(18*y),"Defect_Rate"]
                    c830test.loc[x+3+(18*y),"upward_trend"] = c830test.loc[x+3+(18*y),"Defect_Rate"]

            else:
                pass
        else:
            break

In [30]:
#c830test.upward_trend.fillna(0,inplace=True)
c830 = c830test.copy()

## filter by fleet (C830C)

In [31]:
#filter by C830C
dfptc830c = dfpt.loc[[dfpt.index.get_level_values(0)[-1]]]

#reset index
dfptc830c = dfptc830c.reset_index()

#drop fleet columns
dfptc830c = dfptc830c.drop('Fleet', 1).copy()

#compress columns name
dfptc830c.columns = dfptc830c.columns.map('_'.join)

#set month as index
dfptc830c = dfptc830c.set_index('Record_MonthName_')

#replace all the zero count as 0.1 to prevent NaN error.
dfptc830c = dfptc830c.replace(0.0,0.1)


#C830C mileage
mileage.rename(columns={'Record': 'Record_MonthName_'}, inplace=True)
mileagec830c = mileage.iloc[:,[0,68]]
mileagec830c["C830C (Monthly)"] = mileagec830c["C830C (Monthly)"].fillna(1)
mileagec830c = mileagec830c.set_index('Record_MonthName_')

#fatten mileage columns
mileagec830c.columns = mileagec830c.columns.to_flat_index()

#merge mileage and dfptc830 together
dfptc830c = pd.merge(dfptc830c,mileagec830c, how = "left", left_index=True, right_index=True )

#remove even rows (errenous data merged on every even columns)

#dfptc830c = dfptc830c.iloc[::2]

#calculate defect rate of each fault category
a = dfptc830c.iloc[:,:-1]
b = dfptc830c.iloc[:,-1:]

a = a.fillna(0.1)
b = b.fillna(0.1)

c830c_defectrate = a.div(b.values, axis=0) * 100000.0
c830c_defectrate = c830c_defectrate.round(2)

#calculate mean of each fault category
c830c_mean = c830c_defectrate.tail(18)
c830c_mean = c830c_mean.replace("Train Number_","mean")
c830c_mean.reset_index(inplace=True)
c830c_mean.columns = c830c_mean.columns.str.replace("Train Number_","Mean ")

#calculate std *3 (UCL) of each fault category
c830c_std = (c830c_defectrate.rolling(18).std().tail(18) * 1.645) + (c830c_defectrate.rolling(18).mean().tail(18))
c830c_std = c830c_std.replace("Train Number_","UCL")
c830c_std.reset_index(inplace=True)
c830c_std.columns = c830c_std.columns.str.replace("Train Number_","UCL ")

##fix c830 UCL line by year
fixed_UCL(c830c_std)

#merge mean data and std data
c830c = pd.merge(c830c_mean,c830c_std, how = "left",on  ="Record_MonthName_")
#round the decimal points
c830c = c830c.round(2)

C:\Users\Darius\AppData\Local\Temp/ipykernel_1772/1592562269.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dfptc830c = dfptc830c.drop('Fleet', 1).copy()
C:\Users\Darius\anaconda3\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
C:\Users\Darius\AppData\Local\Temp/ipykernel_1772/1592562269.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mileagec830c["C830C (Monthly)"] = mileagec830c["C830C (Monthly)"].fillna(1)


In [32]:
#melt defect rate columns
c830cdr = pd.melt(c830c, id_vars=['Record_MonthName_'], value_vars=c830c.iloc[:,1:12],var_name='Fault_Category',value_name='Defect_Rate', ignore_index=False)
#generalise fault category labels
c830cdr.Fault_Category = c830cdr.Fault_Category.str.replace("Mean","").str.strip()

#melt defect rate columns
c830cucl = pd.melt(c830c, id_vars=['Record_MonthName_'], value_vars=c830c.iloc[:,12:],var_name='Fault_Category',value_name='UCL', ignore_index=False)

#concat c830dr and c830ucl
c830cdr["UCL"] = c830cucl.iloc[:,2]
c830c = c830cdr
c830c.reset_index(drop=True, inplace=True)

###  check for defect rate above UCL

In [33]:
#check for defect rate above UCL
x=0
c830c["above_UCL"] = ""

for x in range(len(c830c.loc[:,"Defect_Rate"])):
    if c830c.loc[x,"Defect_Rate"] >= c830c.loc[x,"UCL"]:
        c830c.loc[x,"above_UCL"] = c830c.loc[x,"Defect_Rate"]
    else:
        c830c.loc[x,"above_UCL"] = 0

###  loop to capture all upward trends (3 consecutive upward trend)

In [34]:
x=0
y=0

for y in range(c830c.Fault_Category.nunique()):
    for x in range(18):
        if (x+3+(18*y) == 198):
            break
        elif (c830c.loc[x+1+(18*y),"Fault_Category"] == c830c.Fault_Category.unique()[y])|(
            c830c.loc[x+2+(18*y),"Fault_Category"] == c830c.Fault_Category.unique()[y])|(
            c830c.loc[x+3+(18*y),"Fault_Category"] == c830c.Fault_Category.unique()[y]):

            if (c830c.loc[x+(18*y),"Defect_Rate"] < c830c.loc[x+1+(
                18*y),"Defect_Rate"] < c830c.loc[x+2+(18*y),"Defect_Rate"] < c830c.loc[x+3+(18*y),"Defect_Rate"]):

                    c830c.loc[x+(18*y),"upward_trend"] = c830c.loc[x+(18*y),"Defect_Rate"]
                    c830c.loc[x+1+(18*y),"upward_trend"] = c830c.loc[x+1+(18*y),"Defect_Rate"]
                    c830c.loc[x+2+(18*y),"upward_trend"] = c830c.loc[x+2+(18*y),"Defect_Rate"]
                    c830c.loc[x+3+(18*y),"upward_trend"] = c830c.loc[x+3+(18*y),"Defect_Rate"]

            else:
                pass
        else:
            break

In [35]:
c830c.upward_trend.fillna(0,inplace=True)

### combining C830 and C830C df together

In [36]:
c830c["fleet"] = "C830C"
c830["fleet"] = "C830"

train = pd.concat([c830,c830c])

cols = ["above_UCL","upward_trend"]
train[cols] = train[cols].replace({'0':np.nan, 0:np.nan})

### data cleaning (pvw)

In [37]:
#filter to keep only withdrawal data
pvw = pvw.loc[pvw["StockChange Withdrawal"] == "PVW"]

#keep only Train related withdrawal
pvw = pvw.loc[~pvw["Sub-System (RS)"].str.contains("WSS|STP|-|HF")]

#select the required columns
pvw_clean = pvw.loc[:,["Year_Month","Sub-System (RS)","PV","fleet type"]]

#pivot table the dataframe
pvw_gb = pvw_clean.pivot_table(values= "PV",index=["fleet type","Year_Month"],columns="Sub-System (RS)", aggfunc="count").fillna(0.0)

pvw_gb.reset_index(inplace=True)

In [38]:
pvwc830 = pvw_gb.loc[pvw_gb["fleet type"]=="C830"]
pvwc830c = pvw_gb.loc[pvw_gb["fleet type"]=="C830C"]

In [39]:
pvwc830test = pd.melt(pvwc830.iloc[:,1:],id_vars="Year_Month",value_name='pvw_count')
pvwc830ctest = pd.melt(pvwc830c.iloc[:,1:],id_vars="Year_Month",value_name='pvw_count')

In [40]:
pvwc830test.rename(columns={"Year_Month":"Record_MonthName_","Sub-System (RS)":"Fault_Category"}, inplace=True)
pvwc830ctest.rename(columns={"Year_Month":"Record_MonthName_","Sub-System (RS)":"Fault_Category"}, inplace=True)

In [41]:
pvwc830test = pd.merge(pvwc830test,mileagec830, on = ["Record_MonthName_"])
pvwc830ctest = pd.merge(pvwc830ctest,mileagec830c, on = ["Record_MonthName_"])

In [42]:
#normalise withdrawal
#pvwc830n = pvwc830test.pvw_count.div(pvwc830test["C830 (monthly)"].values, axis=0) * 100000.0
#pvwc830cn = pvwc830ctest.pvw_count.div(pvwc830ctest["C830C (Monthly)"].values, axis=0) * 100000.0

pvwc830n = pvwc830test.pvw_count
pvwc830cn = pvwc830ctest.pvw_count

In [43]:
pvwc830n = pd.concat([pvwc830test.iloc[:,0:2], pvwc830n], axis=1)
pvwc830cn = pd.concat([pvwc830ctest.iloc[:,0:2], pvwc830cn], axis=1)

In [44]:
c830 = pd.merge(c830,pvwc830n, how='left' , left_on= ["Record_MonthName_","Fault_Category"], right_on = ["Record_MonthName_","Fault_Category"])
c830c = pd.merge(c830c,pvwc830cn, how='left' , left_on= ["Record_MonthName_","Fault_Category"], right_on = ["Record_MonthName_","Fault_Category"])

In [45]:
c830.pvw_count=c830.pvw_count.replace(0, np.nan)
c830c.pvw_count=c830c.pvw_count.replace(0, np.nan)

### combining C830 and C830C dataframe together

In [46]:
c830c["fleet"] = "C830C"
c830["fleet"] = "C830"

train = pd.concat([c830,c830c])

cols = ["above_UCL","upward_trend"]
train[cols] = train[cols].replace({'0':np.nan, 0:np.nan})

In [47]:
train.Record_MonthName_ = pd.to_datetime(train.Record_MonthName_, format='%Y-%m-%d', errors='ignore').dt.date

### seperate critical and non critical sub-system

In [48]:
train_others = train.loc[train.Fault_Category.str.contains("Driver's Console|Coupler System|Fire Detection System|Gangway|Lighting System")]
train_critical = train.loc[~train.Fault_Category.str.contains("Driver's Console|Coupler System|Fire Detection System|Gangway|Lighting System")]

In [49]:
train_critical.Record_MonthName_ = pd.to_datetime(train_critical.Record_MonthName_)
train_others.Record_MonthName_ = pd.to_datetime(train_others.Record_MonthName_)

C:\Users\Darius\anaconda3\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [57]:
train_critical.to_excel(r"C:\Users\Darius\OneDrive - SMRT Corporation Ltd\RCM\RCM TEMPLATE USE THIS FOR UPDATE_110221\train_critical.xlsx")
train_others.to_excel(r"C:\Users\Darius\OneDrive - SMRT Corporation Ltd\RCM\RCM TEMPLATE USE THIS FOR UPDATE_110221\train_others.xlsx")